# 1. Entendimento do Problema de Negócio

Indaial é uma cidade de Santa Catarina com alto IDH (0.777). Ela tem cerca de <b>70 mil</b> habitantes e conta com <b>13</b> vereadores em sua câmara municipal.

![](imagens/SantaCatarina_Municip_Indaial.png)

Neste projeto vamos analisar as <b>proposições</b> escritas pelos vereadores de Indaial desde <b>1996 até 2023</b>. Ou seja, vamos explorar os projetos de lei, projetos de resolução, requerimentos, indicações ou moções submetidas pelos vereadores para tramitação na Câmara e discussão no plenário.

Isto é importante porque conseguiremos avaliar a participação política dos vereadores eleitos, quais são os temas mais discutidos na câmara, o tempo médio entre uma reunião e uma deliberação, qual a situação geral das proposições e qual é o tema/texto que teve mais aprovação ao longo dos anos.

Este tipo de projeto pode auxiliar aos eleitores a ter um melhor entendimento da gestão pública e do funcionamento da democracia na cidade de Indaial. Além disso, pode facilitar aos meios de comunicação uma busca mais automatizada da atuação dos vereadores.

Para fazer tudo isso, vamos utilizar a linguagem <b>Python</b> para retirar os dados do site da prefeitura de forma automatizada, o <b>SQL</b> para inserir estes registros em um banco de dados e o <b>Power BI</b> para visualizar os insights que podem ser obtidos destes dados.

# 2. Webscrapping do site da Prefeitura de  Indaial

In [1]:
# importando as bibliotecas
import pandas as pd # utilizado para trabalhar com tabelas
import bs4 # utilizado para acessar o código do site da cidade
import requests # fazer as requisições para acessar o código do site da cidade

# variáveis constantes utilizadas ao longo do código
user_agent = ("Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
              "AppleWebKit/537.36 (KHTML, like Gecko)"
              "Chrome/110.0.0.0" 
              "Safari/537.36" 
              "Edg/110.0.1587.63")

Nosso objetivo inicial é acessar cada uma das proposições e obter o código HTML da página.

![](imagens/print_proposicao_162_2023.png)

In [2]:
# url de exemplo de uma proposição do ano de 2023
url = ("https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1"
             "&nrProposicao=162" # Número da proposição
             "&aaProposicao=2023") # Ano da proposição

headers = {'User-Agent': user_agent}

# obtendo o conteúdo html da página
html = requests.get(url, headers = headers, timeout  = 10).content

# passando o html para o BeautifulSoup
soup = bs4.BeautifulSoup(html, 'html.parser')

# imprimindo o código html de forma visual
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <link href="legis.ico" rel="shortcut icon" type="image/x-ico"/>
  <title>
   Câmara Municipal de Indaial _ Indicação nº 162/2023 de 02/03/2023
  </title>
  <meta content="Câmara Municipal de Indaial _ Indicação nº 162/2023 de 02/03/2023" name="description">
   <link href="css/geral3.css" rel="stylesheet"/>
   <link href="https://d11gitgevq44cw.cloudfront.net/libs/font-awesome/5x/css/all.min.css" rel="stylesheet"/>
   <link crossorigin="anonymous" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" rel="stylesheet"/>
   <script crossorigin="anonymous" integrity="sha256-FgpCb/KJQlLNfOu91ta32o/NMZxltwRo8QtmkMRdAu8=" src="https://code.jquery.com/jquery-3.3.1.min.js">
   </script>
   <script crossorigin="anonymous" integrity="sha384-ApNb

In [3]:
# acessando o método findAll do BeatifulSoup
soup.findAll('dt')

[<dt class="col-sm-3">Reunião</dt>,
 <dt class="col-sm-3">Deliberação</dt>,
 <dt class="col-sm-3">Situação</dt>,
 <dt class="col-sm-3">Assunto</dt>,
 <dt class="col-sm-3">Autor</dt>]

In [4]:
# acessando o método findAll do BeatifulSoup
soup.findAll('dd')

[<dd class="col-sm-9">02/03/2023</dd>,
 <dd class="col-sm-9">02/03/2023</dd>,
 <dd class="col-sm-9">Proposição Despachada</dd>,
 <dd class="col-sm-9">Pontes, Bueiros, boca de lobo</dd>,
 <dd class="col-sm-9">Vereador <br/><b>Jessé Cléber de Souza</b>.</dd>]

In [5]:
# colocando os valores em uma tabela do Pandas
cabecalhos = [e.get_text() for e in soup.findAll('dt')] # cria uma lista com o texto de cada uma das dt
df = pd.DataFrame({}, columns = cabecalhos) # criar um dataframe Vazio cujo cabeçalho é a lista obtida acima
df # visualizando o DataFrame criado

,Reunião,Deliberação,Situação,Assunto,Autor


In [6]:
# inserindo os valores em um DataFrame do Pandas
dt = soup.findAll('dt')
dd = soup.findAll('dd')
dic = {}

for i in range(len(dt)):
    x = dt[i].get_text()
    y = dd[i].get_text()
    dic[x] = y
    
dic ={k:[v] for k,v in dic.items()}
dic = pd.DataFrame(dic, columns  = cabecalhos)
df = pd.concat([df,dic], ignore_index=True)
df

,Reunião,Deliberação,Situação,Assunto,Autor
0,02/03/2023,02/03/2023,Proposição Despachada,"Pontes, Bueiros, boca de lobo",Vereador Jessé Cléber de Souza.


### Tornando o webscrapping mais automatizado

In [7]:
# criando um dataframe vazio com os cabeçalhos
df = pd.DataFrame({}, columns = cabecalhos)
df

,Reunião,Deliberação,Situação,Assunto,Autor


In [8]:
proposicao = 1
ano = 2002

url = (f"https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1"
             "&nrProposicao=" + str(proposicao) + # Número da proposição
             "&aaProposicao=" + str(ano)) # Ano da proposição

# obtendo o html da página
html = requests.get(url, headers = headers, timeout = 10).content
# passando o html para o BeautifulSoup
soup = bs4.BeautifulSoup(html, 'html.parser')
# inserindo os valores
dt = soup.findAll('dt')
dd = soup.findAll('dd')
dic = {}

for i in range(len(dt)):
    x = dt[i].get_text()
    y = dd[i].get_text()
    dic[x] = y
    
dic ={k:[v] for k,v in dic.items()}
dic = pd.DataFrame(dic, columns  = cabecalhos)
df = pd.concat([df,dic])
df

,Reunião,Deliberação,Situação,Assunto,Autor
0,01/02/2002,01/02/2002,Proposição Aprovada,Diversos,Vereador Rubens Reinhold Ittner.


### Criando um buscador

In [15]:
import pandas as pd
import datetime
import bs4
import requests
import time

class BuscadorPreposicoesIndaial:
    
    def __init__(self):
        self._cabecalhos = ['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor']
        self._user_agent = ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.63")
        self._df = pd.DataFrame({}, columns = self._cabecalhos)
    
    def getHTML(self, url):
        # obtendo o html da página
        html = requests.get(url, headers = {'User-Agent': self._user_agent}, timeout = 30).content
        # passando o html para o BeautifulSoup
        soup = bs4.BeautifulSoup(html, 'html.parser')
        return soup
    
    def getValuesfromHTML(self,soup):
        # inserindo os valores
        dt = soup.findAll('dt')
        dd = soup.findAll('dd')
        dic = {}

        for i in range(len(dt)):
            x = dt[i].get_text()
            y = dd[i].get_text()
            dic[x] = y

        dic ={k:[v] for k,v in dic.items()}
        dic = pd.DataFrame(dic, columns  = self._cabecalhos)
        return dic
    
    def InserirValores(self,novos_valores):
        self._df = pd.concat([self._df,novos_valores])
    
    def Atualiza(self, data_inicial, data_final):
        
        tempo_inicial = time.time()
        
        for y in range(data_inicial, data_final):
            
            erro = 0
            prop = 1
            
            while erro < 7:
                
                url = (f"https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1"
                         "&nrProposicao=" + str(prop) + # Número da proposição
                         "&aaProposicao=" + str(y)) # Ano da proposição
                
                html = self.getHTML(url)
                
                if html.find(class_='col-10'):
                    erro += 1
                    pass
                else:
                    dic = self.getValuesfromHTML(html)
                    self.InserirValores(dic)
                
                if prop % 400 == 0:
                    tempo_atual = time.time()
                    tempo_transcorrido = round((tempo_atual - tempo_inicial) / 60 , 1)
                    print('Ano: {}, proposição: {}, Tempo: {} min'.format(y,prop, tempo_transcorrido))
                          
                prop += 1
                time.sleep(0.35)

In [16]:
buscador = BuscadorPreposicoesIndaial()
buscador.Atualiza(data_inicial = 1996,data_final = 1998)

KeyboardInterrupt: 

In [17]:
df = buscador._df
df

,Reunião,Deliberação,Situação,Assunto,Autor
0,22/02/1996,22/02/1996,Proposição Aprovada,Serviços e Obras,Vereador Henrique Fritz.
0,22/02/1996,22/02/1996,Proposição Aprovada,Serviços e Obras,Vereador Henrique Fritz.
0,22/02/1996,22/02/1996,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Henrique Fritz.
0,22/02/1996,22/02/1996,Proposição Aprovada,Rede de Água / Esgoto / Pluvial,Vereador Henrique Fritz.
0,22/02/1996,22/02/1996,Proposição Aprovada,Rede de Água / Esgoto / Pluvial,Vereador Henrique Fritz.
...,...,...,...,...,...
0,15/02/1996,15/02/1996,Proposição Aprovada,Iluminação Pública e Rede de Energia Elétrica,Vereador Sílvio Gonçalves da Luz.
0,15/02/1996,15/02/1996,Proposição Aprovada,Serviços e Obras,Vereador Sílvio Gonçalves da Luz.
0,15/02/1996,15/02/1996,Proposição Aprovada,Diversos,Vereador Sílvio Gonçalves da Luz.
0,15/02/1996,15/02/1996,Proposição Aprovada,Diversos,Vereador Sílvio Gonçalves da Luz.


# 3. Inserir os registros em um Banco de Dados

# 4. EDA (Análise Exploratória)

# 5. Aplicação de algum modelo de ML

# 6. Valor Agregado

* o que um empresário ganha com isso? como a população pode se beneficiar?